In [13]:
# =========================
# Cell 1: Install packages
# =========================
!pip install -q "chromadb>=0.5.0,<0.6.0" "sentence-transformers>=3.0.0" "numpy>=1.24.0" groq ipywidgets

print("All dependencies installed!")


# ==================================
# Cell 2: Set up Groq API Key (Colab)
# ==================================
from google.colab import userdata
import os

print("API Key Setup")
print("-" * 50)
print("Get your FREE Groq API key:")
print("1. Visit: https://console.groq.com/")
print("2. Sign up (free)")
print("3. Create API key")
print("4. Paste it below")
print("-" * 50)

# Option 1: Direct input (quick start)
GROQ_API_KEY = input("\n Paste your Groq API key here: ").strip()

# Set environment variable
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

if GROQ_API_KEY:
    print("API Key set successfully!")
else:
    print("No API key provided. Please run this cell again.")


# =========================================
# Cell 3: Updated LLM Engine with Groq (LLM)
# =========================================
from groq import Groq
import os

class LLMEngine:
    """Handles AI model interactions with CURRENT models"""

    def __init__(self):
        self.client = Groq(api_key=os.environ.get('GROQ_API_KEY'))

        # CURRENT WORKING MODELS (October 2025)
        self.working_models = [
            "llama-3.3-70b-versatile",      # Newest, best
            "llama-3.2-90b-text-preview",   # Very powerful
            "llama-3.2-11b-text-preview",   # Balanced
            "llama-3.2-3b-preview",         # Fast
            "mixtral-8x7b-32768",           # Alternative
        ]

        self.model = None
        print(" Finding available model...")

        # Test each model to find working one
        for model_name in self.working_models:
            try:
                test = self.client.chat.completions.create(
                    model=model_name,
                    messages=[{"role": "user", "content": "Hi"}],
                    max_tokens=5
                )
                self.model = model_name
                print(f" Using: {model_name}")
                break
            except Exception as e:
                print(f" {model_name} unavailable")
                continue

        if not self.model:
            print("No Groq models available!")
            print(" SWITCH TO GOOGLE GEMINI (see below)")

    def generate(self, prompt, max_tokens=500, temperature=0.8):
        """Generate response"""

        if not self.model:
            return "ERROR: No working model found. Please switch to Google Gemini."

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are an expert Dungeon Master who creates immersive, engaging fantasy adventures."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].message.content

        except Exception as e:
            error_msg = str(e)

            # If model decommissioned, try next one
            if "decommissioned" in error_msg or "not found" in error_msg:
                print(f" {self.model} decommissioned, trying alternatives...")

                for model_name in self.working_models:
                    if model_name == self.model:
                        continue
                    try:
                        response = self.client.chat.completions.create(
                            model=model_name,
                            messages=[
                                {"role": "system", "content": "You are an expert Dungeon Master."},
                                {"role": "user", "content": prompt}
                            ],
                            temperature=temperature,
                            max_tokens=max_tokens
                        )
                        self.model = model_name
                        print(f" Switched to: {model_name}")
                        return response.choices[0].message.content
                    except:
                        continue

                return "ERROR: All Groq models unavailable. Please switch to Google Gemini."

            return f"Error: {error_msg}"

print(" LLM Engine loaded")


# ==================================
# Cell 4: Memory Management (Hybrid RAG)
# ==================================
import time
import math
import chromadb
from sentence_transformers import SentenceTransformer
from typing import List, Dict

class WorkingMemory:
    """Short-term memory - last N turns"""

    def __init__(self, max_turns=5):
        self.buffer = []
        self.max_turns = max_turns

    def add_turn(self, player_action: str, dm_response: str):
        """Store a game turn"""
        self.buffer.append({
            'turn_number': len(self.buffer) + 1,
            'player': player_action,
            'dm': dm_response,
            'timestamp': time.time()
        })

        if len(self.buffer) > self.max_turns:
            self.buffer.pop(0)

    def get_context(self) -> str:
        """Format recent turns for context"""
        if not self.buffer:
            return "This is the beginning of the adventure."

        context = "Recent events:\n"
        for turn in self.buffer:
            context += f"\nTurn {turn['turn_number']}:\n"
            context += f"Player: {turn['player']}\n"
            context += f"DM: {turn['dm']}\n"
        return context


# Improved PersistentMemory with EphemeralClient + Hybrid Retriever support
class PersistentMemory:
    """Long-term memory with vector store + importance + recency signals"""

    def __init__(self, collection_name: str = "story_memory"):
        print(" Loading memory system (this may take a moment)...")
        # EphemeralClient: no external server; fast and Colab-friendly
        self.client = chromadb.EphemeralClient()

        # Encoder with normalized embeddings for stable cosine similarity
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
        self.encoder.max_seq_length = 256

        # Create/get collection
        try:
            self.collection = self.client.get_collection(collection_name)
        except:
            self.collection = self.client.create_collection(
                collection_name,
                metadata={"hnsw:space": "cosine"}  # cosine similarity
            )

        self.memory_counter = 0
        print("Memory system ready")

    def _next_id(self) -> str:
        idx = self.memory_counter
        self.memory_counter += 1
        return f"memory_{idx:08d}"

    def store_event(
        self,
        event_text: str,
        event_type: str = "general",
        importance: int = 5,
        turn_number: int | None = None,
    ):
        """Store important narrative event with embedding + metadata"""
        text = event_text.strip()
        if not text:
            return

        embedding = self.encoder.encode(text, normalize_embeddings=True).tolist()
        now = time.time()

        self.collection.add(
            embeddings=[embedding],
            documents=[text],
            metadatas=[{
                "type": event_type,
                "importance": int(importance),
                "timestamp": now,
                "turn": int(turn_number) if turn_number is not None else -1
            }],
            ids=[self._next_id()]
        )

    def count(self) -> int:
        return self.collection.count()

    def query_semantic(
        self,
        query: str,
        n_results: int = 8,
        include_documents: bool = True
    ) -> list[dict]:
        """Raw semantic query against the vector store"""
        if self.count() == 0:
            return []

        q = self.encoder.encode(query, normalize_embeddings=True).tolist()
        # Include all useful fields; explicit includes for current Chroma
        result = self.collection.query(
            query_embeddings=[q],
            n_results=min(n_results, self.count()),
            include=["documents", "metadatas", "distances"]
        )

        docs = result.get("documents", [[]])[0]
        metas = result.get("metadatas", [[]])[0]
        dists = result.get("distances", [[]])[0]

        items = []
        for doc, meta, dist in zip(docs, metas, dists):
            items.append({
                "document": doc,
                "metadata": meta or {},
                # cosine distance: smaller = closer. Convert to similarity.
                "semantic_similarity": 1.0 - float(dist)
            })
        return items

    def all_recent(self, limit: int = 30) -> list[dict]:
        """Return up to the last `limit` events by time (no embeddings)"""
        if self.count() == 0:
            return []
        # Fetch and sort client-side
        result = self.collection.get(include=["documents", "metadatas"])
        docs = result.get("documents", [])
        metas = result.get("metadatas", [])
        items = []
        for doc, meta in zip(docs, metas):
            items.append({
                "document": doc,
                "metadata": meta or {},
            })
        items.sort(key=lambda x: x["metadata"].get("timestamp", 0.0), reverse=True)
        return items[:limit]


class HybridRetriever:
    """Hybrid retrieval that balances semantic similarity, recency, and importance"""

    def __init__(self, persistent_memory: PersistentMemory):
        self.persistent = persistent_memory

    @staticmethod
    def _recency_score(ts: float, now: float | None = None, half_life_seconds: float = 3600.0) -> float:
        """
        Exponential time decay: score halves every half_life_seconds.
        Returns [0..1], where 1 is 'just now'.
        """
        if ts is None:
            return 0.0
        if now is None:
            now = time.time()
        age = max(0.0, now - ts)
        return 0.5 ** (age / half_life_seconds)

    @staticmethod
    def _importance_score(importance: int, max_importance: int = 10) -> float:
        imp = max(0, min(int(importance), max_importance))
        return imp / float(max_importance)

    def retrieve(
        self,
        query: str,
        k: int = 6,
        alpha: float = 0.60,   # semantic weight
        beta: float = 0.25,    # recency weight
        gamma: float = 0.15    # importance weight
    ) -> list[dict]:
        """
        Returns top-k items with blended score.
        Score = alpha*semantic + beta*recency + gamma*importance
        """
        # Seed with semantic matches
        sem = self.persistent.query_semantic(query, n_results=max(k * 3, 12))

        # Union with recent items (to ensure recency candidates even if not top semantic)
        recents = self.persistent.all_recent(limit=max(k * 3, 30))

        # Deduplicate by (doc, timestamp)
        def key_of(item: dict) -> tuple:
            return (item.get("document", ""), item.get("metadata", {}).get("timestamp", 0.0))

        merged: dict[tuple, dict] = {}
        for it in sem + recents:
            merged[key_of(it)] = it

        now = time.time()
        scored = []
        for it in merged.values():
            meta = it.get("metadata", {})
            sim = float(it.get("semantic_similarity", 0.0))  # present for semantic items; 0 otherwise
            rec = self._recency_score(meta.get("timestamp"), now)
            imp = self._importance_score(meta.get("importance", 5))
            score = alpha * sim + beta * rec + gamma * imp
            it["score"] = score
            scored.append(it)

        scored.sort(key=lambda x: x["score"], reverse=True)
        return scored[:k]


class MemoryManager:
    """Combined memory system: short-term buffer + hybrid long-term RAG"""

    def __init__(self, working_turns: int = 5):
        self.working = WorkingMemory(max_turns=working_turns)
        self.persistent = PersistentMemory()
        self.retriever = HybridRetriever(self.persistent)

    def add_turn(self, player_action: str, dm_response: str, turn_number: int | None = None):
        """Store turn in both memory systems"""
        self.working.add_turn(player_action, dm_response)
        # Extract and store salient events with turn number for better traceability
        self.extract_and_store_events(dm_response, turn_number=turn_number)

    def extract_and_store_events(self, dm_response: str, turn_number: int | None = None):
        """Extract important events from response (simple heuristic)"""
        important_keywords = [
            'discovered', 'found', 'met', 'killed', 'obtained', 'learned',
            'entered', 'left', 'promised', 'swore', 'agreed', 'revealed',
            'reveals', 'rescued', 'lost', 'won', 'unlocked', 'deciphered',
            'solved', 'ambushed', 'betrayed', 'allied', 'vowed'
        ]

        response_lower = dm_response.lower()
        # Always store a short summary with moderate importance as a fallback
        short = dm_response.split('\n')[0][:200].strip()
        if short:
            self.persistent.store_event(short, event_type="summary", importance=5, turn_number=turn_number)

        # Store sentences containing important keywords with higher importance
        sentences = [s.strip() for s in dm_response.replace('!', '.').replace('?', '.').split('.') if s.strip()]
        for sentence in sentences:
            s_lower = sentence.lower()
            if any(kw in s_lower for kw in important_keywords):
                self.persistent.store_event(sentence, event_type="event", importance=7, turn_number=turn_number)

    def build_full_context(self, current_query: str, rag_k: int = 6) -> str:
        """Combine working + long-term memories into a concise context"""
        # Short-term transcript
        context = self.working.get_context()

        # Long-term relevant snippets (hybrid)
        retrieved = self.retriever.retrieve(current_query, k=rag_k)
        if retrieved:
            context += "\n\nRelevant past events:\n"
            for item in retrieved:
                snippet = item["document"]
                meta = item.get("metadata", {})
                turn = meta.get("turn", -1)
                tag = meta.get("type", "memory")
                context += f"- ({tag}, turn {turn}) {snippet}\n"

        return context

    def test_query(self, query: str, k: int = 6):
        """Helper for UI memory search"""
        hits = self.retriever.retrieve(query, k=k)
        lines = []
        for i, h in enumerate(hits, 1):
            meta = h.get("metadata", {})
            lines.append(f"{i}. (score={h['score']:.2f}, turn={meta.get('turn', -1)}, imp={meta.get('importance', 5)}) {h['document']}")
        return "\n".join(lines) if lines else "No memories found. Play more turns to build memory!"

print(" Memory system loaded")


# =============================================
# Cell 5: IMPROVED Player Profiler (unchanged)
# =============================================
class PlayerProfiler:
    """Analyzes and adapts to player behavior"""

    def __init__(self):
        self.profile = {
            'total_turns': 0,
            'combat_turns': 0,
            'dialogue_turns': 0,
            'exploration_turns': 0,
            'puzzle_turns': 0,
            'combat_oriented': 0.0,
            'roleplay_focused': 0.0,
            'exploration_driven': 0.0,
            'puzzle_solver': 0.0,
            'avg_action_length': 0,
            'questions_asked': 0,
        }

    def analyze_action(self, player_action: str):
        """Update profile based on player's action"""
        self.profile['total_turns'] += 1

        action_type = self._classify_action(player_action)
        self._update_playstyle_scores()
        self._update_engagement(player_action)

    def _classify_action(self, action: str) -> str:
        """Categorize the action with IMPROVED keywords"""
        action_lower = action.lower()

        combat_kw = [
            'attack', 'fight', 'strike', 'cast', 'shoot', 'defend', 'kill',
            'hit', 'punch', 'stab', 'slash', 'dodge', 'parry', 'charge',
            'swing', 'fire', 'aim', 'block', 'counter', 'rage', 'smash'
        ]
        if any(kw in action_lower for kw in combat_kw):
            self.profile['combat_turns'] += 1
            return 'combat'

        dialogue_kw = [
            'ask', 'tell', 'talk', 'say', 'persuade', 'convince', 'speak',
            'greet', 'introduce', 'chat', 'discuss', 'question', 'answer',
            'reply', 'respond', 'converse', 'negotiate', 'argue', 'debate',
            'whisper', 'shout', 'call out', 'mention', 'explain'
        ]
        if any(kw in action_lower for kw in dialogue_kw):
            self.profile['dialogue_turns'] += 1
            return 'dialogue'

        explore_kw = [
            'search', 'investigate', 'explore', 'examine', 'look', 'check',
            'walk', 'run', 'move', 'go', 'enter', 'leave', 'exit', 'climb',
            'jump', 'cross', 'follow', 'approach', 'retreat', 'flee', 'escape',
            'wander', 'travel', 'venture', 'navigate', 'head', 'proceed',
            'step', 'advance', 'return', 'descend', 'ascend', 'direction'
        ]
        if any(kw in action_lower for kw in explore_kw):
            self.profile['exploration_turns'] += 1
            return 'exploration'

        puzzle_kw = [
            'solve', 'figure out', 'decode', 'think', 'analyze', 'decipher',
            'calculate', 'reason', 'deduce', 'ponder', 'consider', 'study',
            'read', 'inspect closely', 'observe', 'notice', 'detect'
        ]
        if any(kw in action_lower for kw in puzzle_kw):
            self.profile['puzzle_turns'] += 1
            return 'puzzle'

        self.profile['exploration_turns'] += 1
        return 'exploration'

    def _update_playstyle_scores(self):
        """Calculate playstyle percentages"""
        total = max(self.profile['total_turns'], 1)
        self.profile['combat_oriented'] = self.profile['combat_turns'] / total
        self.profile['roleplay_focused'] = self.profile['dialogue_turns'] / total
        self.profile['exploration_driven'] = self.profile['exploration_turns'] / total
        self.profile['puzzle_solver'] = self.profile['puzzle_turns'] / total

    def _update_engagement(self, action: str):
        """Track engagement metrics"""
        word_count = len(action.split())
        total = self.profile['total_turns']
        current_avg = self.profile['avg_action_length']

        self.profile['avg_action_length'] = (
            (current_avg * (total - 1) + word_count) / total
        )

        if '?' in action:
            self.profile['questions_asked'] += 1

    def get_dominant_style(self) -> tuple:
        """Return (style_name, score)"""
        styles = {
            'Combat-Oriented': self.profile['combat_oriented'],
            'Roleplay-Focused': self.profile['dialogue_turns'],
            'Exploration-Driven': self.profile['exploration_turns'],
            'Puzzle-Solver': self.profile['puzzle_turns']
        }
        # Fix dominant calculation to use normalized percentages
        styles = {
            'Combat-Oriented': self.profile['combat_oriented'],
            'Roleplay-Focused': self.profile['roleplay_focused'],
            'Exploration-Driven': self.profile['exploration_driven'],
            'Puzzle-Solver': self.profile['puzzle_solver']
        }
        dominant = max(styles.items(), key=lambda x: x[1])
        return dominant

    def get_adaptive_guidance(self) -> str:
        """Provide guidance for story adaptation"""
        style, score = self.get_dominant_style()

        if score < 0.2:
            return "Player is exploring their style. Offer varied content."

        guidance = {
            'Combat-Oriented': "Include tactical encounters and action scenes.",
            'Roleplay-Focused': "Emphasize NPC interactions and moral choices.",
            'Exploration-Driven': "Present mysteries, hidden areas, and discoveries.",
            'Puzzle-Solver': "Include riddles, logic challenges, and investigation."
        }
        return guidance.get(style, "Balanced content mix.")

    def display_profile(self):
        """Show player profile"""
        print("\n" + "="*60)
        print(" PLAYER PROFILE ANALYSIS")
        print("="*60)
        print(f"Total Turns: {self.profile['total_turns']}")
        print(f"\nPlaystyle Breakdown:")
        print(f"  Combat:      {'█' * int(self.profile['combat_oriented'] * 20)} {self.profile['combat_oriented']:.1%}")
        print(f"  Roleplay:    {'█' * int(self.profile['roleplay_focused'] * 20)} {self.profile['roleplay_focused']:.1%}")
        print(f"  Exploration: {'█' * int(self.profile['exploration_driven'] * 20)} {self.profile['exploration_driven']:.1%}")
        print(f"  Puzzle:      {'█' * int(self.profile['puzzle_solver'] * 20)} {self.profile['puzzle_solver']:.1%}")

        style, score = self.get_dominant_style()
        print(f"\n Dominant Style: {style} ({score:.1%})")
        print(f" Adaptation: {self.get_adaptive_guidance()}")
        print(f" Avg Action Length: {self.profile['avg_action_length']:.1f} words")
        print(f" Questions Asked: {self.profile['questions_asked']}")

        print(f"\nAction Breakdown:")
        print(f"  Combat: {self.profile['combat_turns']} turns")
        print(f"  Dialogue: {self.profile['dialogue_turns']} turns")
        print(f"  Exploration: {self.profile['exploration_turns']} turns")
        print(f"  Puzzle: {self.profile['puzzle_turns']} turns")
        print("="*60 + "\n")

print("Player Profiler loaded (IMPROVED)")


# ======================================
# Cell 6: Dungeon Master (hybrid memory)
# ======================================
class DungeonMaster:
    """The AI Dungeon Master - main controller"""

    def __init__(self):
        print(" Initializing AI Dungeon Master...")
        self.llm = LLMEngine()
        self.memory = MemoryManager()
        self.profiler = PlayerProfiler()  # Uses new improved profiler
        self.turn_count = 0
        self.story_started = False
        print(" Dungeon Master ready!\n")

    def start_adventure(self):
        opening_prompt = """Create an engaging opening for a fantasy adventure.
        Requirements:
        - Set a vivid scene
        - Introduce an immediate hook
        - Keep it concise (3-4 sentences)
        - End with a question or prompt for action
        Create the opening:"""

        opening = self.llm.generate(opening_prompt, temperature=0.9)
        # Store the opening as an important long-term event (turn 0)
        self.memory.persistent.store_event(
            f"Adventure began: {opening[:200]}",
            event_type="opening",
            importance=9,
            turn_number=0
        )
        self.story_started = True
        return opening

    def process_turn(self, player_action: str) -> str:
        self.turn_count += 1
        self.profiler.analyze_action(player_action)

        # Build RAG context (short-term + hybrid long-term)
        context = self.memory.build_full_context(player_action)

        player_guidance = self.profiler.get_adaptive_guidance()

        dm_prompt = f"""You are an expert Dungeon Master.

=== CONTEXT ===
Turn: {self.turn_count}
{context}

=== ADAPTATION ===
{player_guidance}

=== PLAYER ACTION ===
{player_action}

Respond naturally (3-5 sentences), stay consistent, keep story moving:"""

        dm_response = self.llm.generate(dm_prompt, temperature=0.8, max_tokens=400)

        # Store memories with turn annotation
        self.memory.add_turn(player_action, dm_response, turn_number=self.turn_count)
        return dm_response

    def show_player_profile(self):
        self.profiler.display_profile()

    def test_memory_recall(self, query: str):
        print(f"\n Memory search: '{query}'")
        print("\n Retrieved memories:")
        print(self.memory.test_query(query, k=8))
        print()

print(" DungeonMaster reloaded")


# ==========================================
# Cell 7: Interactive Game Interface (Colab)
# ==========================================
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

class ColabGameInterface:
    """Interactive game interface for Colab"""

    def __init__(self):
        self.dm = DungeonMaster()
        self.game_active = False

        # Create UI elements
        self.output = widgets.Output()
        self.input_box = widgets.Text(
            placeholder='Enter your action...',
            description='Action:',
            layout=widgets.Layout(width='80%')
        )
        self.submit_btn = widgets.Button(
            description='Submit Action',
            button_style='success',
            layout=widgets.Layout(width='15%')
        )
        self.profile_btn = widgets.Button(
            description='Show Profile',
            button_style='info'
        )
        self.memory_btn = widgets.Button(
            description='Test Memory',
            button_style='warning'
        )

        # Memory test input
        self.memory_input = widgets.Text(
            placeholder='e.g., bartender, forest, bridge',
            description='Query:',
            layout=widgets.Layout(width='60%')
        )
        self.memory_submit_btn = widgets.Button(
            description='Search Memory',
            button_style='info',
            layout=widgets.Layout(width='20%')
        )

        self.start_btn = widgets.Button(
            description=' Start New Adventure',
            button_style='primary',
            layout=widgets.Layout(width='50%', height='50px')
        )

        # Bind events
        self.submit_btn.on_click(self.on_submit)
        self.profile_btn.on_click(self.on_profile)
        self.memory_btn.on_click(self.on_memory_toggle)
        self.memory_submit_btn.on_click(self.on_memory_search)
        self.start_btn.on_click(self.on_start)

        # Memory test UI visibility
        self.memory_ui_visible = False

    def on_start(self, b):
        """Start the game"""
        with self.output:
            clear_output()
            print("╔" + "="*58 + "╗")
            print("║" + " "*15 + "AI DUNGEON MASTER" + " "*26 + "║")
            print("║" + " "*10 + "An Adaptive Storytelling Experience" + " "*13 + "║")
            print("╚" + "="*58 + "╝\n")

            print(" DM: Let me set the scene...\n")
            opening = self.dm.start_adventure()
            print(f" {opening}\n")
            print("-" * 60 + "\n")

        self.game_active = True
        self.start_btn.layout.visibility = 'hidden'

    def on_submit(self, b):
        """Handle player action"""
        if not self.game_active:
            with self.output:
                print(" Please start the adventure first!")
            return

        action = self.input_box.value.strip()
        if not action:
            return

        with self.output:
            print(f" You: {action}\n")
            print(" DM: ", end="", flush=True)

            response = self.dm.process_turn(action)
            print(f"{response}\n")
            print("-" * 60 + "\n")

            # Show milestone
            if self.dm.turn_count % 10 == 0:
                print(f"[Milestone: Turn {self.dm.turn_count}! Analyzing playstyle...]\n")
                self.dm.show_player_profile()

        self.input_box.value = ""

    def on_profile(self, b):
        """Show player profile"""
        with self.output:
            self.dm.show_player_profile()

    def on_memory_toggle(self, b):
        """Toggle memory search UI"""
        self.memory_ui_visible = not self.memory_ui_visible
        if self.memory_ui_visible:
            with self.output:
                print("\n Memory Search Mode Activated")
                print("Enter a query below and click 'Search Memory'\n")
        else:
            with self.output:
                print("\n Memory Search Mode Closed\n")

    def on_memory_search(self, b):
        """Test memory recall"""
        query = self.memory_input.value.strip()
        if not query:
            with self.output:
                print(" Please enter a search query")
            return

        with self.output:
            self.dm.test_memory_recall(query)

        self.memory_input.value = ""

    def display(self):
        """Display the interface"""
        # Start button
        display(self.start_btn)

        # Input area
        input_area = widgets.HBox([self.input_box, self.submit_btn])
        display(input_area)

        # Control buttons
        controls = widgets.HBox([self.profile_btn, self.memory_btn])
        display(controls)

        # Memory search UI (always visible input, user controls via button text prompts)
        memory_area = widgets.HBox([self.memory_input, self.memory_submit_btn])
        display(memory_area)

        # Output area
        display(self.output)

print(" Game interface loaded")

# ===============================================
# ChromaDB Memory Inspector - Add as new cell
# ===============================================

class MemoryInspector:
    """Inspect and debug ChromaDB long-term memory"""

    def __init__(self, memory_manager):
        self.memory = memory_manager.persistent
        self.collection = self.memory.collection

    def show_stats(self):
        """Display database statistics"""
        print("\n" + "="*60)
        print("CHROMADB MEMORY STATISTICS")
        print("="*60)
        print(f"Total memories stored: {self.memory.count()}")
        print(f"Collection name: {self.collection.name}")
        print(f"Memory counter: {self.memory.memory_counter}")
        print("="*60 + "\n")

    def show_all_memories(self, limit=20):
        """Display all stored memories with metadata"""
        if self.memory.count() == 0:
            print(" No memories stored yet. Play some turns first!\n")
            return

        print("\n" + "="*60)
        print(f" ALL MEMORIES (showing up to {limit})")
        print("="*60 + "\n")

        # Fetch all data
        result = self.collection.get(
            include=["documents", "metadatas", "embeddings"]
        )

        docs = result.get("documents", [])
        metas = result.get("metadatas", [])
        ids = result.get("ids", [])
        embeddings = result.get("embeddings", [])

        # Display each memory
        for i, (doc, meta, mem_id, emb) in enumerate(zip(docs[:limit], metas[:limit], ids[:limit], embeddings[:limit]), 1):
            print(f"Memory #{i}")
            print(f"  ID: {mem_id}")
            print(f"  Type: {meta.get('type', 'unknown')}")
            print(f"  Turn: {meta.get('turn', -1)}")
            print(f"  Importance: {meta.get('importance', 5)}/10")
            print(f"  Timestamp: {meta.get('timestamp', 0)}")
            print(f"  Embedding dimensions: {len(emb) if emb else 0}")
            print(f"  Content: {doc[:150]}{'...' if len(doc) > 150 else ''}")
            print("-" * 60)

        if len(docs) > limit:
            print(f"\n... and {len(docs) - limit} more memories")
        print()

    def show_recent_memories(self, n=10):
        """Display most recent memories"""
        if self.memory.count() == 0:
            print("No memories stored yet.\n")
            return

        print("\n" + "="*60)
        print(f"MOST RECENT {n} MEMORIES")
        print("="*60 + "\n")

        recent = self.memory.all_recent(limit=n)

        for i, item in enumerate(recent, 1):
            meta = item.get("metadata", {})
            doc = item.get("document", "")

            import time
            ts = meta.get('timestamp', 0)
            time_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ts))

            print(f"{i}. Turn {meta.get('turn', -1)} | {meta.get('type', 'memory')} | Importance: {meta.get('importance', 5)}")
            print(f"   Time: {time_str}")
            print(f"   {doc[:200]}{'...' if len(doc) > 200 else ''}")
            print("-" * 60)
        print()

    def show_by_type(self):
        """Show memories grouped by type"""
        if self.memory.count() == 0:
            print(" No memories stored yet.\n")
            return

        print("\n" + "="*60)
        print(" MEMORIES BY TYPE")
        print("="*60 + "\n")

        result = self.collection.get(include=["documents", "metadatas"])
        metas = result.get("metadatas", [])

        # Group by type
        by_type = {}
        for meta in metas:
            mem_type = meta.get('type', 'unknown')
            by_type[mem_type] = by_type.get(mem_type, 0) + 1

        for mem_type, count in sorted(by_type.items(), key=lambda x: x[1], reverse=True):
            print(f"  {mem_type}: {count} memories")
        print()

    def show_by_importance(self):
        """Show distribution of importance scores"""
        if self.memory.count() == 0:
            print(" No memories stored yet.\n")
            return

        print("\n" + "="*60)
        print(" IMPORTANCE DISTRIBUTION")
        print("="*60 + "\n")

        result = self.collection.get(include=["metadatas"])
        metas = result.get("metadatas", [])

        # Count by importance
        by_importance = {}
        for meta in metas:
            imp = meta.get('importance', 5)
            by_importance[imp] = by_importance.get(imp, 0) + 1

        for imp in sorted(by_importance.keys(), reverse=True):
            count = by_importance[imp]
            bar = "█" * (count // 2 + 1)
            print(f"  {imp}/10: {bar} ({count})")
        print()

    def test_semantic_search(self, query, n=5):
        """Test semantic search with detailed results"""
        print("\n" + "="*60)
        print(f" SEMANTIC SEARCH: '{query}'")
        print("="*60 + "\n")

        if self.memory.count() == 0:
            print(" No memories stored yet.\n")
            return

        results = self.memory.query_semantic(query, n_results=n)

        if not results:
            print("No matching memories found.\n")
            return

        for i, item in enumerate(results, 1):
            doc = item.get("document", "")
            meta = item.get("metadata", {})
            sim = item.get("semantic_similarity", 0.0)

            print(f"{i}. Similarity: {sim:.3f} | Turn: {meta.get('turn', -1)} | Type: {meta.get('type', 'unknown')}")
            print(f"   {doc[:200]}{'...' if len(doc) > 200 else ''}")
            print("-" * 60)
        print()

    def show_memory_by_turn(self, turn_number):
        """Show all memories from a specific turn"""
        print("\n" + "="*60)
        print(f" MEMORIES FROM TURN {turn_number}")
        print("="*60 + "\n")

        if self.memory.count() == 0:
            print(" No memories stored yet.\n")
            return

        result = self.collection.get(include=["documents", "metadatas"])
        docs = result.get("documents", [])
        metas = result.get("metadatas", [])

        found = []
        for doc, meta in zip(docs, metas):
            if meta.get('turn') == turn_number:
                found.append((doc, meta))

        if not found:
            print(f"No memories found for turn {turn_number}.\n")
            return

        for i, (doc, meta) in enumerate(found, 1):
            print(f"{i}. Type: {meta.get('type', 'unknown')} | Importance: {meta.get('importance', 5)}")
            print(f"   {doc}")
            print("-" * 60)
        print()

    def full_diagnostic(self):
        """Run complete diagnostic"""
        print("\n" + " RUNNING FULL DIAGNOSTIC" + "\n")
        self.show_stats()
        self.show_by_type()
        self.show_by_importance()
        self.show_recent_memories(5)



# ========================================
# Quick Check Function
# ========================================

def quick_memory_check(game_or_dm):
    """Quick check if memories are being saved"""

    if hasattr(game_or_dm, 'dm'):
        dm = game_or_dm.dm
    else:
        dm = game_or_dm

    count = dm.memory.persistent.count()
    turn = dm.turn_count

    print("\n" + "="*60)
    print(" QUICK MEMORY CHECK")
    print("="*60)
    print(f"Current turn: {turn}")
    print(f"Memories stored: {count}")
    print(f"Expected memories: ~{turn * 2} (2 per turn average)")

    if count == 0:
        print("\n WARNING: No memories stored!")
        print(" TIP: Play at least 3-5 turns to build memory")
    elif count < turn:
        print("\n  Few memories stored (this is normal)")
        print(" TIP: Important events get stored automatically")
    else:
        print("\n Memory system working correctly!")

    print("="*60 + "\n")

    # Show a sample memory if any exist
    if count > 0:
        recent = dm.memory.persistent.all_recent(limit=1)
        if recent:
            print(" Most recent memory:")
            print(f"   {recent[0]['document'][:150]}...")
            print()


# ========================================
# Interactive Inspector Widget
# ========================================

def create_inspector_ui(game_or_dm):
    """Create interactive inspection UI"""
    import ipywidgets as widgets
    from IPython.display import display, clear_output

    if hasattr(game_or_dm, 'dm'):
        dm = game_or_dm.dm
    else:
        dm = game_or_dm

    inspector = MemoryInspector(dm.memory)
    output = widgets.Output()

    # Buttons
    stats_btn = widgets.Button(description=" Stats", button_style='info')
    recent_btn = widgets.Button(description=" Recent", button_style='success')
    types_btn = widgets.Button(description=" By Type", button_style='warning')
    search_btn = widgets.Button(description=" Search", button_style='primary')
    full_btn = widgets.Button(description="Full Check", button_style='danger')

    search_box = widgets.Text(
        placeholder='Enter search query...',
        description='Search:',
        layout=widgets.Layout(width='50%')
    )

    def on_stats(b):
        with output:
            clear_output()
            inspector.show_stats()

    def on_recent(b):
        with output:
            clear_output()
            inspector.show_recent_memories(10)

    def on_types(b):
        with output:
            clear_output()
            inspector.show_by_type()
            inspector.show_by_importance()

    def on_search(b):
        query = search_box.value.strip()
        if query:
            with output:
                clear_output()
                inspector.test_semantic_search(query, n=8)

    def on_full(b):
        with output:
            clear_output()
            inspector.full_diagnostic()

    stats_btn.on_click(on_stats)
    recent_btn.on_click(on_recent)
    types_btn.on_click(on_types)
    search_btn.on_click(on_search)
    full_btn.on_click(on_full)

    buttons = widgets.HBox([stats_btn, recent_btn, types_btn, full_btn])
    search_area = widgets.HBox([search_box, search_btn])

    display(buttons)
    display(search_area)
    display(output)

    # Show initial stats
    with output:
        inspector.show_stats()


print(" ChromaDB Inspector loaded!")
print("\n Usage:")
print("  inspector = inspect_memory(game.dm)")
print("  quick_memory_check(game.dm)")
print("  create_inspector_ui(game.dm)  # Interactive UI")
# ===========================
# Cell 8: Start Playing (UI)
# ===========================
print(" Launching AI Dungeon Master...\n")

game = ColabGameInterface()
game.display()

print("\n Tips:")
print("1. Click 'Start New Adventure' to begin")
print("2. Type your actions in the text box")
print("3. Click 'Show Profile' to see your playstyle analysis")
print("4. Click 'Test Memory' and enter queries to check long-term recall")


# ======================================
# Cell 9: Automated Demo (Optional test)
# ======================================
def run_demo():
    """Run automated demo"""
    print("\n RUNNING AUTOMATED DEMO\n")

    dm = DungeonMaster()

    # Start
    print("="*60)
    opening = dm.start_adventure()
    print(f" DM: {opening}\n")

    # Test actions
    test_actions = [
        "I look around the room carefully",
        "I approach the bartender and ask about rumors",
        "I order an ale and sit down to listen",
        "I examine the mysterious hooded figure in the corner",
        "I ask the figure about the ancient ruins",
        "I agree to help retrieve the lost artifact",
        "I prepare my gear and head towards the forest",
        "I search for tracks or signs of danger",
        "I encounter a wolf - I try to communicate peacefully",
        "I continue deeper into the forest",
        "I find ancient stone markers",
        "I investigate the ruins entrance",
    ]

    for i, action in enumerate(test_actions, 1):
        print(f"\n{'='*60}")
        print(f"Turn {i}/{len(test_actions)}")
        print(f"{'='*60}")
        print(f"  Player: {action}")
        response = dm.process_turn(action)
        print(f" DM: {response}")

    # Final analysis
    print("\n" + "="*60)
    print(" DEMO COMPLETE - Final Analysis")
    print("="*60)
    dm.show_player_profile()

    # Test memory
    print("\n Testing Long-Term Memory:")
    dm.test_memory_recall("bartender")
    dm.test_memory_recall("ruins")

    return dm

# Uncomment to run demo
# demo_dm = run_demo()


# ==============================================
# Cell 10: Save and Load Game State (Optional)
# ==============================================
import pickle
from google.colab import files

def save_game(dm, filename="dungeon_master_save.pkl"):
    """Save game state"""
    with open(filename, 'wb') as f:
        pickle.dump({
            'turn_count': dm.turn_count,
            'profiler': dm.profiler.profile,
            'working_memory': dm.memory.working.buffer
        }, f)

    # Download to local machine
    files.download(filename)
    print(f" Game saved to {filename}")

def load_game(dm, filename="dungeon_master_save.pkl"):
    """Load game state"""
    # Upload from local machine
    uploaded = files.upload()

    with open(filename, 'rb') as f:
        data = pickle.load(f)
        dm.turn_count = data['turn_count']
        dm.profiler.profile = data['profiler']
        dm.memory.working.buffer = data['working_memory']

    print(f" Game loaded from {filename}")


# ============================
# Bonus: Character Tracker
# ============================
class CharacterTracker:
    def __init__(self):
        self.characters = {}

    def add_character(self, name, description, turn_met):
        self.characters[name] = {
            'description': description,
            'first_met': turn_met,
            'interactions': [],
            'relationship': 'neutral'
        }

    def get_character_context(self, name):
        if name in self.characters:
            char = self.characters[name]
            return f"{name}: Met at turn {char['first_met']}, {len(char['interactions'])} interactions"
        return f"{name}: New character"

# Example usage in DungeonMaster (if desired):
# self.character_tracker = CharacterTracker()


# ======================
# Bonus: Quest System
# ======================
class QuestLog:
    def __init__(self):
        self.quests = []

    def add_quest(self, title, description):
        self.quests.append({
            'title': title,
            'description': description,
            'status': 'active',
            'progress': []
        })

    def display_quests(self):
        print("\n ACTIVE QUESTS:")
        for i, quest in enumerate(self.quests, 1):
            if quest['status'] == 'active':
                print(f"{i}. {quest['title']} - {quest['description']}")

# Example usage in DungeonMaster (if desired):
# self.quest_log = QuestLog()


# ===========================
# Final: Ready with fixes
# ===========================
game = ColabGameInterface()
game.display()
print("\n Game ready with hybrid RAG memory!")
print(" 'Test Memory' now shows ranked results with scores and turns.")

All dependencies installed!
API Key Setup
--------------------------------------------------
Get your FREE Groq API key:
1. Visit: https://console.groq.com/
2. Sign up (free)
3. Create API key
4. Paste it below
--------------------------------------------------

 Paste your Groq API key here: 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
API Key set successfully!
 LLM Engine loaded
 Memory system loaded
Player Profiler loaded (IMPROVED)
 DungeonMaster reloaded
 Game interface loaded
 ChromaDB Inspector loaded!

 Usage:
  inspector = inspect_memory(game.dm)
  quick_memory_check(game.dm)
  create_inspector_ui(game.dm)  # Interactive UI
 Launching AI Dungeon Master...

 Initializing AI Dungeon Master...
 Finding available model...


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


 Using: llama-3.3-70b-versatile
 Loading memory system (this may take a moment)...
Memory system ready
 Dungeon Master ready!



Button(button_style='primary', description=' Start New Adventure', layout=Layout(height='50px', width='50%'), …

Output()

...

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


 Using: llama-3.3-70b-versatile
 Loading memory system (this may take a moment)...
Memory system ready
 Dungeon Master ready!



Button(button_style='primary', description=' Start New Adventure', layout=Layout(height='50px', width='50%'), …

Output()


 Game ready with hybrid RAG memory!
 'Test Memory' now shows ranked results with scores and turns.


In [14]:
# Quick check (easiest)
quick_memory_check(game.dm)


 QUICK MEMORY CHECK
Current turn: 0
Memories stored: 29
Expected memories: ~0 (2 per turn average)

 Memory system working correctly!

 Most recent memory:
   The phrase "seek the truth in the embers" suggests that you may need to search for clues or remnants of something in a place where a fire has been ext...



In [15]:
# Detailed inspection
inspector = inspect_memory(game.dm)


🔍 CHROMADB MEMORY INSPECTOR

Available commands:
  1. inspector.show_stats()
  2. inspector.show_all_memories()
  3. inspector.show_recent_memories(10)
  4. inspector.show_by_type()
  5. inspector.show_by_importance()
  6. inspector.test_semantic_search('your query')
  7. inspector.show_memory_by_turn(5)
  8. inspector.full_diagnostic()


 RUNNING FULL DIAGNOSTIC


CHROMADB MEMORY STATISTICS
Total memories stored: 29
Collection name: story_memory
Memory counter: 0


 MEMORIES BY TYPE

  summary: 16 memories
  event: 12 memories
  opening: 1 memories


 IMPORTANCE DISTRIBUTION

  9/10: █ (1)
  7/10: ███████ (12)
  5/10: █████████ (16)


MOST RECENT 5 MEMORIES

1. Turn 16 | event | Importance: 7
   Time: 2025-10-09 19:21:10
   The phrase "seek the truth in the embers" suggests that you may need to search for clues or remnants of something in a place where a fire has been extinguished, and you start to suspect that the villa...
------------------------------------------------------------

In [16]:
# Launch interactive inspector
create_inspector_ui(game.dm)

Output()

In [17]:
# Quick fix - run this cell immediately
inspector.show_all_memories = lambda limit=20: print("Loading...") or _show_all_memories_fixed(inspector, limit)

def _show_all_memories_fixed(inspector, limit=20):
    """Fixed version of show_all_memories"""
    if inspector.memory.count() == 0:
        print(" No memories stored yet. Play some turns first!\n")
        return

    print("\n" + "="*60)
    print(f" ALL MEMORIES (showing up to {limit})")
    print("="*60 + "\n")

    result = inspector.collection.get(
        include=["documents", "metadatas", "embeddings"]
    )

    docs = result.get("documents", [])
    metas = result.get("metadatas", [])
    ids = result.get("ids", [])
    embeddings = result.get("embeddings", [])

    for i, (doc, meta, mem_id, emb) in enumerate(zip(docs[:limit], metas[:limit], ids[:limit], embeddings[:limit]), 1):
        print(f"Memory #{i}")
        print(f"  ID: {mem_id}")
        print(f"  Type: {meta.get('type', 'unknown')}")
        print(f"  Turn: {meta.get('turn', -1)}")
        print(f"  Importance: {meta.get('importance', 5)}/10")
        print(f"  Timestamp: {meta.get('timestamp', 0)}")
        # Fixed: proper check for embeddings
        emb_len = len(emb) if emb is not None else 0
        print(f"  Embedding dimensions: {emb_len}")
        print(f"  Content: {doc[:150]}{'...' if len(doc) > 150 else ''}")
        print("-" * 60)

    if len(docs) > limit:
        print(f"\n... and {len(docs) - limit} more memories")
    print()

# Now try again
inspector.show_all_memories(limit=20)

Loading...

 ALL MEMORIES (showing up to 20)

Memory #1
  ID: memory_00000000
  Type: opening
  Turn: 0
  Importance: 9/10
  Timestamp: 1760032008.1857374
  Embedding dimensions: 384
  Content: Adventure began: As the last wisps of sunlight faded beyond the mist-shrouded mountains, the village of Brindlemark lay bathed in an eerie, flickering...
------------------------------------------------------------
Memory #2
  ID: memory_00000001
  Type: summary
  Turn: 1
  Importance: 5/10
  Timestamp: 1760032345.537999
  Embedding dimensions: 384
  Content: As you step into the village of Brindlemark, the villagers' nervous glances fall upon you, their faces illuminated by the dancing flames of the torche...
------------------------------------------------------------
Memory #3
  ID: memory_00000002
  Type: summary
  Turn: 2
  Importance: 5/10
  Timestamp: 1760032384.6247494
  Embedding dimensions: 384
  Content: Thorne's eyes narrow slightly, as if sizing you up, before he nods and gestures f